<a href="https://colab.research.google.com/github/holocenecap/holocene-colab/blob/main/Copy_of_%5B3%5D_Aave's_Best_Pools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
#Imports & settings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set_theme(style="ticks", color_codes=True)
%matplotlib inline
#%load_ext google.colab.data_table
%reload_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))

dplyr.summarise.inform,[RTYPES.NILSXP]


In [2]:
#@title

%%R
#Grab base query from Flipside
df = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/d47d0ebc-6ace-4b42-800e-df8206dc9d4c/data/latest', simplifyDataFrame = TRUE)

#Change the date to date format
#df$BLOCK_TIMESTAMP <- parse_datetime(df$BLOCK_TIMESTAMP)
df$WEEK <- parse_datetime(df$WEEK)

#Rename & reorder columns
df <- df %>% rename(week = WEEK, 
                   aave_market = AAVE_MARKET, reserve_name = RESERVE_NAME,
                   total_liquidity_token = TOTAL_LIQUIDITY_TOKEN, total_liquidity_usd = TOTAL_LIQUIDITY_USD,
                   utilization_rate = UTILIZATION_RATE, supply_rate = SUPPLY_RATE,
                   flash_premium = FLASH_PREMIUM, flash_premium_usd = FLASH_PREMIUM_USD,
                   interest_received_usd = INTEREST_RECEIVED_USD, total_returns_usd = TOTAL_RETURNS_USD,
                   overall_apr = OVERALL_APR) %>%
  mutate(utilization_rate = utilization_rate * 100,
         overall_apr = overall_apr * 100,
         supply_rate = supply_rate * 100) %>%
  select(week, aave_market, reserve_name, total_liquidity_token, total_liquidity_usd,
         utilization_rate, supply_rate, flash_premium, flash_premium_usd,
         interest_received_usd, total_returns_usd, overall_apr)

#drop the markets without a full 6 month history
df <- df %>% 
  left_join(df %>% group_by(aave_market) %>% tally(), by = "aave_market") %>%
  filter(n == 26) %>%
  select(-n)


#Find the biggest by overall APR & fees
rank_table <- df %>% 
  group_by(aave_market, reserve_name) %>%
  summarise(mean_apr = mean(overall_apr),
            total_fees = sum(total_returns_usd),
            mean_liq = mean(total_liquidity_usd)) %>%
  ungroup() %>%
  arrange(desc(mean_apr)) %>%
  mutate(apr_rank = row_number(desc(mean_apr)),
         total_fee_rank = row_number(desc(total_fees)))

#Join this back to the main table
df <- df %>%
  left_join(rank_table %>% select (aave_market, apr_rank, total_fee_rank), by="aave_market")

#add weekly ranks to main table by APR & fees
df <- df %>%
  group_by(week) %>%
  mutate(week_apr_rank = row_number(desc(overall_apr)),
         week_fee_rank = row_number(desc(total_returns_usd))) %>%
  ungroup()


# Aave - Ranking the Top 5 Pools

Aave is a borrowing & lending protocol, which operates a series of pools, each dedicated to single token.  Each pool in Aave accepts deposits from users, and lends these deposits back to other users.  As it doesn't make a lot of sense to deposit an asset & take a loan of the same asset, the Aave magic comes from being able to borrow a different asset to the one you deposited.  This has a number of advantages & use cases:
- You need stablecoins to yield farm but don't want to sell your ETH?  Deposit your eth, borrow stables.
- Long-short exposure - it's May 2021 and your charts tell you ETH is about to crash by 60%.  Deposit stables & borrow ETH.  Pay the loan back at half price after the crash. Vice versa if you're bullish on ETH.
- Parking long term HODL funds in a safe place for a bit of deposit yield in return.

Aave pools pay fees to depositors to reward them for providing liquidity in the pool.  The Aave V2 pools with the highest fees over the last 6 months are shown in the chart below.  This chart displays the ranking of each pool over each week, with the 5 pools selected being the ones with the highest fees returned over the entire 6 months.  The USDC pool is consistently the highest, with the DAI and USDT pool trading 2nd and 3rd place regularly.  In 4th place is the WETH pool, and the BUSD pool was in the top 5 for a while before losing ground to other tokens.

In [3]:
#@title
#Top 5 pools by week for 6 months
df_p = %R df %>% filter(total_fee_rank <= 5) %>% arrange(week, reserve_name)

fig = px.line(df_p, x="week", y="week_fee_rank", color='reserve_name',
              template="simple_white", width=800, height=800/1.618, 
              title= 'Top 5 Pools by Total Fees - Ranking by Fees Earned',
              labels=dict(week="Week", week_fee_rank="Rank", reserve_name="Pool"))
fig.update_yaxes(autorange="reversed")
fig.update_traces(mode="lines+markers")
fig.update_yaxes(tick0=1, dtick=1)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=0.01
))
fig.show()

## How close are the pools to each other?

To understand how close the pools are in terms of fees, the table below shows the total fees over the 6 month period for each of these 5 pools.  The USDC pool is not just the largest pool, it is the largest by a factor of almost 3.  We suspected the USDT & DAI pools were close, and the data confirms it - less than 10% difference in fees over the period.  We will examine USDT & DAI a bit closer later.  The fees from WETH and BUSD are significantly lower than than the pools above.  An interesting point for these two pools is the average liquidity deposited in the pool over the 6 month period - WETH generated almost twice the fees but required 137 times more liquidity to do it.  More on this later as well...

In [4]:
#@title
#What are the top 5 pools?
#- by total fees
from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:,.0f}"
#%load_ext google.colab.data_table
df_p = %R rank_table %>% filter(total_fee_rank <= 5) %>% arrange(desc(total_fees)) %>% select(reserve_name, total_fees, mean_liq) %>% rename("Market Symbol" = reserve_name, "Total Fees USD" = total_fees, "Average Liquidity USD" = mean_liq)
data_table.DataTable(df_p, include_index=True, min_width = "1")

,Market Symbol,Total Fees USD,Average Liquidity USD
1,USDC,4.775151e+07,2.900649e+09
2,USDT,1.724878e+07,7.154147e+08
3,DAI,1.583000e+07,8.883746e+08
4,WETH,1.077217e+06,2.330169e+09
5,BUSD,6.139782e+05,1.700161e+07


## Comparing DAI & USDT

As we saw from the ranking table, USDT and DAI are very close together with respect to fees generated.  The graph below shows this in more detail - the fees received by each pool on each week. An interesting point is the week of the 9th of August - USDT receieved 800k more fees than DAI in just this week. This difference accounts for a large portion of the overall difference between USDT and DAI for the 6 month period (800k out of 1400k).

In [5]:
#@title
#Compare DAI & USDT
df_p = %R df %>% filter(reserve_name %in% c("USDT", "DAI")) %>% arrange(week, reserve_name)

fig = px.line(df_p, x="week", y="total_returns_usd", color='reserve_name',
              template="simple_white", width=800, height=800/1.618, 
              title= 'DAI & USDT Total USD Fees Received - Head to Head',
              labels=dict(week="Week", total_returns_usd="USD Fees Received", reserve_name="Pool"))
#fig.update_yaxes(autorange="reversed")
#fig.update_traces(mode="lines+markers")
#fig.update_yaxes(tick0=1, dtick=1)
fig.update_traces(line_shape = "spline", line_smoothing=.7, selector=dict(type='scatter'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.show()


# Why are the results like this?

The basic Aave pool model is that borrowers pay an interest rate for the funds they borrow, and depositors receive a portion of this interest rate as a reward for providing liquidity.  Users can choose between a fixed interest rate and a variable rate.  The variable rate is determined by a couple of factors, the most important being the pool utiisation rate.  For each pool, this is the defined as:

> $UtilisationRate = \frac{BorrowedFunds}{DepositedFunds}$

The higher the utilisation rate, the higher the interest rate.  The interest rate curve is non-linear - there is an inflection point called the Optimal Utilisation rate - $U_{Optimal}$.  For utilsation rates below $U_{Optimal}$, the interest rate changes slowly with utilisation rate.  For rates above $U_{Optimal}$, interest rates increase sharply.  This mechanism helps to ensure that there is always sufficient liquidity in the pool for depositors to retrieve their funds.  If the pool is stretched beyond $U_{Optimal}$, very high interest rates will discourage borrowers and encourage further deposits until things come back into balance.  Different tokens have different $U_{Optimal}$ and different interest rate slope parameters.  There is a more detailed explanation of this mechanism, along with the current parameter values for each token at [https://docs.aave.com/risk/liquidity-risk/borrow-interest-rate](https://docs.aave.com/risk/liquidity-risk/borrow-interest-rate).

The digression above into the detail of the interest rate model is relevant to our investigation, as it helps to explain why some pools return higher fees than others.  There are three factors which drive the fees in an Aave pool:
- The interest rate for deposits (driven by pool utilisation rate)
- The size of the pool - interest earned = pool liquidity x interest rate
- The flashloan fees received by the pool (advanced users & protocols can borrow & repay loans in the same block without depositing collateral - for a small fee which goes to the pool depositors)

To understand why the 5 pools above are ranked like they are, we will delve into these three factors.


## Deposit Interest Rate

The chart below shows the deposit interest rate for the top 5 pools discussed above.  Notice that the stablecoins are all highly correlated - they are consitently with a few % of each other.  There are a few outlier points for BUSD which will be covered later.  There is an overall declining trend for the stablecoins- this indicates a lower demand for borrowing stablecoins from March through to June.  This coincided with the boom & bust in crypto prices.  There has been a slight recovery at the beginning of August, again coinciding with increased asset prices.  

The major outlier on the graph, however, is WETH.  The deposit interest rate is extremely low compared with the stablecoins, and is also relatively stable.  We will look at the reasons behind this when we examine the pool utilisation rates.

In [6]:
#@title
#Plot APR by pool
df_p = %R df %>% filter(total_fee_rank <= 5) %>% arrange(week, reserve_name)

fig = px.line(df_p, x="week", y="supply_rate", color='reserve_name',
              template="simple_white", width=800, height=800/1.618, 
              title= 'Top 5 Pools by Total Fees - Deposit Interest Rate',
              labels=dict(week="Week", supply_rate="Deposit Interest Rate %", reserve_name="Pool"))
#fig.update_yaxes(autorange="reversed")
#fig.update_traces(mode="lines+markers")
#fig.update_yaxes(tick0=1, dtick=1)
fig.update_traces(line_shape = "spline", line_smoothing=.7, selector=dict(type='scatter'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.show()

## Pool Utilisation Rate

We discussed the impact of pool utilisation rate on deposit interest rate.  Below is the graph of this in action, and it helps to explain the observations in the interest rate chart above.  Firstly we see that WETH has a very low utilisation rate.  Currently less than 5% of the pool is loaned out, and it reached a high of only 20%.  It seems like people don't want to borrow ETH - this makes sense as stablecoin yields in farms are usually much higher than volatile assets.

BUSD is the other interesting character.  You'll remember the spikes in deposit rate above - these coincide with the times when BUSD utilisation rate creeps up to and above 80%.  From the [Aave docs](https://docs.aave.com/risk/liquidity-risk/borrow-interest-rate), we see that the inflection point for the BUSD rate curve is 80%, and the ramp up rate above 80% is much higher than USDC, USDT or DAI - BUSD at 100% versus the others at 60% and 75%.  These parameters make BUSD interest rates respond much more aggressively when utilisation rates rise, compared with the other stablecoins.  This is an important factor in how BUSD, a relatively minor stablecoin with low total liquidity, is in the top 5 fee generators on Aave.

In [7]:
#@title
#Plot Util by pool
df_p = %R df %>% filter(total_fee_rank <= 5) %>% arrange(week, reserve_name)

fig = px.line(df_p, x="week", y="utilization_rate", color='reserve_name',
              template="simple_white", width=800, height=800/1.618, 
              title= 'Top 5 Pools by Total Fees - Utilisation Rate',
              labels=dict(week="Week", utilization_rate="Utilisation Rate", reserve_name="Pool"))
#fig.update_yaxes(autorange="reversed")
#fig.update_traces(mode="lines+markers")
fig.update_yaxes(tick0=10, dtick=10)
fig.update_traces(line_shape = "spline", line_smoothing=.7, selector=dict(type='scatter'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.5,
    xanchor="right",
    x=0.99
))
fig.show()

## Pool Liquidity

The fees received by a pool are both a function of the interest rate and the total deposited liquidity in the pool.  The graph below shows the total pool liquidity for the 5 pools in our study.  Here we see why USDC is the dominant fee collecting pool on Aave.  It currently holds nearly USD 6b of liquidity, more than triple its closest stablecoin rival. It has consistently held more liquidity than the other stablecoins.  As shown above, the interest rate for USDC is comparable to the other stablecoins, hence the main driver for USDC being in the number 1 position on the fee leaderboard is the size of the pool.

We also get to see the impact of interest rate on returns.  WETH only came in at number 4 on the leaderboard, but below we can see it has as much liquidity as USDC.  The low returns on WETH deposited (driven by the low utilisation rate) means that total fees generated by this pool are much lower.

In [8]:
#@title
#Plot liquidity by pool
df_p = %R df %>% filter(total_fee_rank <= 5) %>% arrange(week, reserve_name)

fig = px.line(df_p, x="week", y="total_liquidity_usd", color='reserve_name',
              template="simple_white", width=800, height=800/1.618, 
              title= 'Top 5 Pools by Total Fees - Pool Liquidity',
              labels=dict(week="Week", total_liquidity_usd="Liquidity USD", reserve_name="Pool"))
#fig.update_yaxes(autorange="reversed")
#fig.update_traces(mode="lines+markers")
#fig.update_yaxes(tick0=1, dtick=1)
fig.update_traces(line_shape = "spline", line_smoothing=.7, selector=dict(type='scatter'))
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

## Flashloan Fees

Flashloans are a feature provided by Aave whereby users can take an uncollateralised loan, provided it is paid back within the same Ethereum block.  There is only a programmatic interface to the flashloan feature, so it tends to be used by other protocols or by advanced users such as bot developers.  Flashloans collect fees for the pool from where the funds are borrowed - 0.9% of the total borrowed) - which accrue to the depositors in the pool.

The graph below shows the total fee income by week for the top 5 pools, split between interest and flashloan income.  For the stablecoin pools, flashloans are a relatively small portion of the total income - USDC is the only one with any discernable flashloan fees in a few weeks.  The WETH pool however is more interesting.  The big spike in fees in May 2021 was almost entirely due to flashloans.  This coincided with the crash in crypto asset prices, and was probably due to flashloans being used to liquidate insolvent positions in protocols such as Aave and Compound.

In [9]:
#@title
#Stacked bar of fees split with interest & flashloan
#Plot subplots of ytoken price & ROI
df_p = %R df %>% filter(total_fee_rank <= 5) %>% arrange(week, reserve_name)

names = %R rank_table %>% arrange(total_fee_rank) %>% head(5) %>% select(reserve_name)
fig = make_subplots(rows=5, cols=1, subplot_titles=(names['reserve_name'].tolist()))
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==1]["week"], y=df_p[df_p.total_fee_rank==1]["interest_received_usd"], name = "Interest"), row=1, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==1]["week"], y=df_p[df_p.total_fee_rank==1]["flash_premium_usd"], name = "Flashloan Fees"), row=1, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==2]["week"], y=df_p[df_p.total_fee_rank==2]["interest_received_usd"], name = "Interest"), row=2, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==2]["week"], y=df_p[df_p.total_fee_rank==2]["flash_premium_usd"], name = "Flashloan Fees"), row=2, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==3]["week"], y=df_p[df_p.total_fee_rank==3]["interest_received_usd"], name = "Interest"), row=3, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==3]["week"], y=df_p[df_p.total_fee_rank==3]["flash_premium_usd"], name = "Flashloan Fees"), row=3, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==4]["week"], y=df_p[df_p.total_fee_rank==4]["interest_received_usd"], name = "Interest"), row=4, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==4]["week"], y=df_p[df_p.total_fee_rank==4]["flash_premium_usd"], name = "Flashloan Fees"), row=4, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==5]["week"], y=df_p[df_p.total_fee_rank==5]["interest_received_usd"], name = "Interest"), row=5, col=1)
fig.append_trace(go.Bar(x=df_p[df_p.total_fee_rank==5]["week"], y=df_p[df_p.total_fee_rank==5]["flash_premium_usd"], name = "Flashloan Fees"), row=5, col=1)
#fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["vault_7day_ROI"], name="ROI"), row=2, col=1)
fig.update_layout(width=800, height=1000, title_text="Fees Generated by Pool")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Fees in USD')
fig.update_layout(barmode='stack')
#fig.update_yaxes(title_text='ROI % Annualised', row=2, col=1)
fig.show()

# Best Pool to Invest In?

If you are a depositor in Aave, then putting your money in the pool which generates the most fees may not get you the best return.  You will be looking for the pool with the highest rate of return - fees divided by liquidity.  The table below shows the top 20 pools arranged by Annualised Percentage Return (APR).

Looking for our previous top 5 pools, we see that they are no longer top of the heap.  BUSD emerges as the winner, returning 6.8% annualised on average over the 6 months.  USDT and DAI managed to stay in the top 5, but lost ground relative to their previous positions.  USDC has dropped to 7th overall based on return - unsurprising given the amount of capital deployed to this pool.  Looking right to the bottom of our list, in 19th place, we find WETH.  With a very low deposit rate, and a very high total liquidity, it's unurprising that the overal yield from this pool is only just higher than zero.

In [10]:
#@title
#What are the top 5 pools?
#- by return
#from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:,.2f}"
#%load_ext google.colab.data_table
df_p = %R rank_table %>% filter(apr_rank <= 20) %>% arrange(desc(mean_apr)) %>% select(reserve_name, total_fees, mean_liq, mean_apr) %>% rename("Market Symbol" = reserve_name, "Total Fees USD" = total_fees, "Average Liquidity USD" = mean_liq, "Average %APR" = mean_apr)
data_table.DataTable(df_p, include_index=True, min_width = "1")

,Market Symbol,Total Fees USD,Average Liquidity USD,Average %APR
1,BUSD,6.139782e+05,1.700161e+07,6.804760
2,GUSD,5.200082e+05,1.944919e+07,6.552753
3,USDT,1.724878e+07,7.154147e+08,6.197722
4,SUSD,4.361157e+05,3.682283e+07,5.647608
5,DAI,1.583000e+07,8.883746e+08,5.564751
6,TUSD,6.011458e+05,4.580273e+07,5.547570
7,USDC,4.775151e+07,2.900649e+09,5.434317
8,SNX,3.400481e+05,1.367232e+07,5.238995
9,KNC,1.035214e+05,3.698451e+06,3.691047
10,CRV,4.194324e+05,3.041755e+07,3.310814


# Conclusions

We have explored the top 5 pools by total fees paid in the Aave V2 protocol.   We have dived into the factors which drive pool fees - pool parameters set by the protocol, pool utilisation rate, pool liquidity and flashloan activity.  We also explored how high fees generated don't necessarily mean high returns for users, and overall pool %APR is a better measure of returns for a user.